In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import csv

# Get the list of NASDAQ companies
nasdaq_tickers = pd.read_csv('nasdaq_screener.csv')
tickers_total = nasdaq_tickers['Symbol'].tolist()
tickers = tickers_total[20:30]

# Prepare the CSV file
csv_file = 'nasdaq_financial_data.csv'

# Helper function to get the stock prices over a range of dates
def get_stock_prices(ticker, start_date, end_date):
    try:
        stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)
        return stock_data['Close']
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.Series()

# Compute the stock value ratio (item_stock%/sp500_stock%) in a year
def get_stock_value_ratio(stock_prices, sp500_prices, date):
    try:
        if date not in stock_prices or date not in sp500_prices:
            return None
        date_diff = date - timedelta(weeks=52)
        if date_diff not in stock_prices or date_diff not in sp500_prices:
            return None
        stock_change = stock_prices[date] / stock_prices[date_diff]
        sp500_change = sp500_prices[date] / sp500_prices[date_diff]
        return stock_change / sp500_change
    except Exception as e:
        print(f"Error computing ratio for date {date}: {e}")
        return None

# Flatten the data of a DataFrame
def flatten_data(df, ticker):
    df_u = df.unstack().reset_index()
    df_u['Ticker'] = df_u.apply(lambda row: f"{row['level_0'].year} {row['level_1']}", axis=1)
    df_u.set_index('Ticker', inplace=True)
    df_ordered = df_u.drop(columns=['level_0', 'level_1'])
    df_transposed = df_ordered.T
    df_transposed['Ticker'] = ticker
    return df_transposed

# Function to get stock values and ratios for a DataFrame
def get_stock_values_and_ratios(df, ticker, stock_prices, sp500_prices):
    for date in df.columns:
        date_str = date.strftime('%Y-%m-%d')
        stock_value = stock_prices.get(date_str, None)
        ratio = get_stock_value_ratio(stock_prices, sp500_prices, date)
        df.at['Stock value', date] = stock_value
        df.at['Ratio ticker sp500', date] = ratio
    return df

# Create the complete DataFrame for all tickers
def create_df(tickers):
    df_complete = pd.DataFrame()
    sp500_prices = get_stock_prices('^GSPC', datetime.now() - timedelta(weeks=52), datetime.now())
    
    def process_ticker(ticker):
        try:
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            income_statement = stock.financials
            cashflow_statement = stock.cashflow
            
            stock_prices = get_stock_prices(ticker, datetime.now() - timedelta(weeks=52), datetime.now())
            df_bs = get_stock_values_and_ratios(balance_sheet, ticker, stock_prices, sp500_prices)
            df_bs_f = flatten_data(df_bs, ticker)
            df_is_f = flatten_data(income_statement, ticker)
            df_cs_f = flatten_data(cashflow_statement, ticker)
            df_row = pd.concat([df_bs_f, df_is_f, df_cs_f], axis=1)
            return df_row
        except Exception as e:
            print(f"Error processing ticker {ticker}: {e}")
            return pd.DataFrame()
    
    with ThreadPoolExecutor() as executor:
        results = executor.map(process_ticker, tickers)
        for result in results:
            if not result.empty:
                df_complete = pd.concat([df_complete, result])
    
    return df_complete

# Generate the complete DataFrame for selected tickers
df_result = create_df(tickers)
df_result
#df_result.to_csv(csv_file)


ABLLW: Period '1mo' is invalid, must be one of ['1d', '5d']


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

# Get the list of NASDAQ companies
nasdaq_tickers = pd.read_csv('nasdaq_screener.csv')
tickers_total = nasdaq_tickers['Symbol'].tolist()
tickers = tickers_total[20:30]

# Prepare the CSV file
csv_file = 'nasdaq_financial_data.csv'

# Helper function to get the stock prices over a range of dates
def get_stock_prices(ticker, start_date, end_date):
    try:
        stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date, interval='1d')
        return stock_data['Close']
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.Series()

# Compute the stock value ratio (item_stock%/sp500_stock%) in a year
def get_stock_value_ratio(stock_prices, sp500_prices, date):
    try:
        if date not in stock_prices or date not in sp500_prices:
            return None
        date_diff = date - timedelta(weeks=52)
        if date_diff not in stock_prices or date_diff not in sp500_prices:
            return None
        stock_change = stock_prices[date] / stock_prices[date_diff]
        sp500_change = sp500_prices[date] / sp500_prices[date_diff]
        return stock_change / sp500_change
    except Exception as e:
        print(f"Error computing ratio for date {date}: {e}")
        return None

# Flatten the data of a DataFrame
def flatten_data(df, ticker):
    df = df.loc[:, ~df.columns.duplicated()]  # Remove duplicated columns
    df_u = df.unstack().reset_index()
    df_u['Ticker'] = df_u.apply(lambda row: f"{row['level_0'].year} {row['level_1']}", axis=1)
    df_u.set_index('Ticker', inplace=True)
    df_ordered = df_u.drop(columns=['level_0', 'level_1'])
    df_transposed = df_ordered.T
    df_transposed['Ticker'] = ticker
    return df_transposed

# Function to get stock values and ratios for a DataFrame
def get_stock_values_and_ratios(df, ticker, stock_prices, sp500_prices):
    for date in df.columns:
        if isinstance(date, str):
            date = datetime.strptime(date, '%Y-%m-%d')
        date_str = date.strftime('%Y-%m-%d')
        stock_value = stock_prices.get(date_str, None)
        ratio = get_stock_value_ratio(stock_prices, sp500_prices, date)
        df.at['Stock value', date] = stock_value
        df.at['Ratio ticker sp500', date] = ratio
    return df

# Create the complete DataFrame for all tickers
def create_df(tickers):
    df_complete = pd.DataFrame()
    sp500_prices = get_stock_prices('^GSPC', datetime.now() - timedelta(weeks=52), datetime.now())
    
    def process_ticker(ticker):
        try:
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            income_statement = stock.financials
            cashflow_statement = stock.cashflow
            
            # Remove duplicate indices if any
            balance_sheet = balance_sheet.loc[~balance_sheet.index.duplicated(keep='first')]
            income_statement = income_statement.loc[~income_statement.index.duplicated(keep='first')]
            cashflow_statement = cashflow_statement.loc[~cashflow_statement.index.duplicated(keep='first')]

            stock_prices = get_stock_prices(ticker, datetime.now() - timedelta(weeks=52), datetime.now())
            df_bs = get_stock_values_and_ratios(balance_sheet, ticker, stock_prices, sp500_prices)
            df_bs_f = flatten_data(df_bs, ticker)
            df_is_f = flatten_data(income_statement, ticker)
            df_cs_f = flatten_data(cashflow_statement, ticker)
            df_row = pd.concat([df_bs_f, df_is_f, df_cs_f], axis=1)
            return df_row
        except Exception as e:
            print(f"Error processing ticker {ticker}: {e}")
            return pd.DataFrame()
    
    with ThreadPoolExecutor() as executor:
        results = executor.map(process_ticker, tickers)
        for result in results:
            if not result.empty:
                df_complete = pd.concat([df_complete, result])
    
    return df_complete

# Generate the complete DataFrame for selected tickers
df_result = create_df(tickers)
df_result.to_csv(csv_file)


ABLLW: Period '1mo' is invalid, must be one of ['1d', '5d']


InvalidIndexError: Reindexing only valid with uniquely valued Index objects